<a href="https://colab.research.google.com/github/kattens/AutoEncoders-for-Protein-Sequences/blob/main/Interaction_and_Pathways_PDB_retrival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.8 MB/s eta 0:00:00


In [5]:
import Bio
import csv
import pandas as pd
import os

In [6]:
interaction_folder_path = '/content/drive/MyDrive/IDS_Target_Result_Interaction'

In [8]:
import os
import pandas as pd

# Define your directory path
interaction_folder_path = '/content/drive/MyDrive/IDS_Target_Result_Interaction'

# List to store empty CSV file names
empty_csv_list = []

# Iterate through files in the directory
for file in os.listdir(interaction_folder_path):
    file_path = os.path.join(interaction_folder_path, file)

    # Skip if the path is a directory
    if os.path.isdir(file_path):
        continue

    # Read the CSV and check if it is empty
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            empty_csv_list.append(file)
            os.remove(file_path)
            print(f"Removed empty file: {file}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

print("Empty files found and removed:", empty_csv_list)


Removed empty file: 5330175.csv
Removed empty file: 5311340.csv
Removed empty file: 221354.csv
Removed empty file: 5329480.csv
Removed empty file: 6806409.csv
Removed empty file: 12947.csv
Removed empty file: 135421339.csv
Removed empty file: 53464483.csv
Removed empty file: 9810709.csv
Removed empty file: 3647519.csv
Removed empty file: 6413301.csv
Removed empty file: 53315868.csv
Removed empty file: 24756910.csv
Removed empty file: 6918097.csv
Removed empty file: 73416445.csv
Removed empty file: 4534086.csv
Removed empty file: 132928.csv
Removed empty file: 5281035.csv
Removed empty file: 9852185.csv
Removed empty file: 51000408.csv
Removed empty file: 73602827.csv
Removed empty file: 3499.csv
Removed empty file: 9809926.csv
Removed empty file: 41867.csv
Removed empty file: 24858111.csv
Removed empty file: 2798.csv
Removed empty file: 33630.csv
Removed empty file: 676352.csv
Removed empty file: 49855250.csv
Removed empty file: 24795070.csv
Removed empty file: 11978790.csv
Removed emp

In [9]:
len(empty_csv_list)

39

In [13]:
cdot_excel = '/content/drive/MyDrive/cdot_actives_50.xlsx'
df = pd.read_excel(cdot_excel)
df.head()


,CDoT Plate No.,*.1 Barcode_Well address,Unnamed: 2,Hit rate,pert_iname,pert_iname.1,concentration_mM,vendor,catalog_no,smiles,...,pubchem_cid,clinical_phase,moa,target,disease_area,indication,DMPNN_prediction,max_tanimoto_active,MAIP_pred_score,max_topological_tanimoto
0,19,BR00130947K18,BRD-K51329597-001-02-9,85.069661,-AZMÂ 475271.00,-AZMÂ 475271.00,10.000000,Tocris,3963,COc1ccc(Cl)c(Nc2ncnc3cc(OCC4CCN(C)CC4)c(OC)cc2...,...,5330175.0,Preclinical,SRC inhibitor,SRC,NaN,NaN,1.0,0.458333,75.354837,0.768537
1,11,BR00130917J06,BRD-K17705806-003-03-6,100.303774,JTC-801,JTC-801,10.000000,Selleck,S2722,CCc1ccc(OCc2ccccc2C(=O)Nc2ccc3nc(C)cc(N)c3c2)cc1,...,5311340.0,Phase 2,opioid receptor antagonist,OPRL1,NaN,NaN,1.0,0.538462,42.501938,0.706559
2,12,BR00130916N19,BRD-K57169635-001-05-9,94.288306,dacomitinib,dacomitinib,10.000000,Tocris,6231,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)\C=C\C...,...,11511120.0,Launched,EGFR inhibitor,EGFR|ERBB2|ERBB4,NaN,NaN,1.0,0.393617,80.514961,0.615859
3,20,BR00130945P03,BRD-K73799155-001-01-5,97.792777,NSC-5844,NSC-5844,10.000048,MedChemEx,HY-100033,Clc1ccc2c(NCCNc3ccnc4cc(Cl)ccc34)ccnc2c1,...,221354.0,Preclinical,CC chemokine receptor agonist,CCR1,NaN,NaN,1.0,0.966667,52.324497,1.000000
4,14,BR00130957L17,BRD-K26122255-001-01-7,94.788143,VLX600,VLX600,10.000000,Enamine,EN300-395210,CC(=NNc1nnc2c(n1)[nH]c1c(C)cccc21)c1ccccn1,...,6806409.0,Phase 1,antitumor agent|ubiquitin C-terminal hydrolase...,TP53|USP14,NaN,NaN,1.0,0.385965,14.558880,0.730836


In [21]:
import requests
import os
import pandas as pd

# Load PubChem IDs from a DataFrame (ensure the column is named 'pubchem_cid')
cdot_excel = '/content/drive/MyDrive/cdot_actives_50.xlsx'
df = pd.read_excel(cdot_excel)

# Remove rows with NaN values in the 'pubchem_cid' column
df = df.dropna(subset=['pubchem_cid'])

# Convert PubChem IDs to integers to avoid formatting issues
pubchem_ids = df['pubchem_cid'].astype(int)

# Output directory for PDB files
output_dir = '/content/drive/MyDrive/PDB_Interaction'
os.makedirs(output_dir, exist_ok=True)

# RCSB API base URL for PDB file download
pdb_download_url = 'https://files.rcsb.org/download/{}.pdb'

# Function to search RCSB using PubChem ID and download PDB files
def search_and_download_pdb(pubchem_id):
    query = {
        "query": {
            "type": "terminal",
            "service": "text_chem",
            "parameters": {
                "attribute": "rcsb_chem_comp_related.resource_accession_code",
                "operator": "exact_match",
                "value": str(pubchem_id)
            }
        },
        "return_type": "entry",
        "request_options": {
            "paginate": {
                "start": 0,
                "rows": 100
            }
        }
    }

    url = 'https://search.rcsb.org/rcsbsearch/v2/query'
    response = requests.post(url, json=query)

    if response.status_code == 200:
        data = response.json()
        pdb_ids = [result['identifier'] for result in data.get('result_set', [])]

        if pdb_ids:
            print(f'Found PDB entries for PubChem ID {pubchem_id}: {pdb_ids}')
            for pdb_id in pdb_ids:
                download_pdb_file(pubchem_id, pdb_id)
        else:
            print(f'No PDB entries found for PubChem ID {pubchem_id}.')
    else:
        print(f'Error {response.status_code} for PubChem ID {pubchem_id}.')

# Function to download PDB files into a folder named after the PubChem ID
def download_pdb_file(pubchem_id, pdb_id):
    download_url = pdb_download_url.format(pdb_id)
    response = requests.get(download_url)

    if response.status_code == 200:
        pubchem_folder = os.path.join(output_dir, str(pubchem_id))
        os.makedirs(pubchem_folder, exist_ok=True)

        pdb_path = os.path.join(pubchem_folder, f'{pdb_id}.pdb')

        with open(pdb_path, 'w') as f:
            f.write(response.text)
        print(f'Downloaded {pdb_id}.pdb to {pubchem_folder}')
    else:
        print(f'Failed to download {pdb_id}.pdb')

# Run search and download for all PubChem IDs
for pubchem_id in pubchem_ids:
    search_and_download_pdb(pubchem_id)


Error 204 for PubChem ID 5330175.
Error 204 for PubChem ID 5311340.
Found PDB entries for PubChem ID 11511120: ['4I23', '4I24']
Downloaded 4I23.pdb to /content/drive/MyDrive/PDB_Interaction/11511120
Downloaded 4I24.pdb to /content/drive/MyDrive/PDB_Interaction/11511120
Error 204 for PubChem ID 221354.
Error 204 for PubChem ID 6806409.
Error 204 for PubChem ID 5329480.
Error 204 for PubChem ID 12947.
Found PDB entries for PubChem ID 444810: ['1CET', '4FGL', '4V2O', '7DI7', '8F4Z']
Downloaded 1CET.pdb to /content/drive/MyDrive/PDB_Interaction/444810
Downloaded 4FGL.pdb to /content/drive/MyDrive/PDB_Interaction/444810
Downloaded 4V2O.pdb to /content/drive/MyDrive/PDB_Interaction/444810
Downloaded 7DI7.pdb to /content/drive/MyDrive/PDB_Interaction/444810
Downloaded 8F4Z.pdb to /content/drive/MyDrive/PDB_Interaction/444810
Error 204 for PubChem ID 135421339.
Found PDB entries for PubChem ID 9939609: ['5I9I']
Downloaded 5I9I.pdb to /content/drive/MyDrive/PDB_Interaction/9939609
Found PDB ent